In [15]:
import itertools as it
from itertools import combinations_with_replacement as cwr
from functools import lru_cache, reduce
import numpy as np
from math import gcd, comb

!pip install -q selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
# this function is used in conjuction with the string conds while playing a game
# given a = [x,y,z], it returns a vector over all possible boards of whether the value at
#     position (x,y) is equal to z
def add(a):
    return sols[:,int(a[0]),int(a[1])] == int(a[2])

In [1]:
# this class is used to represent all of the possible solutions for a row/col
# it uses a trie to make membership checks easier to implement
# it may not be too useful overall though
class Group:
    def __init__(self):
        # a dict storing the actual trie structure
        self.tree = {}
        # a list of the strings stored in the trie
        self.vals = []
        
    def __len__(self):
        return len(self.vals)
    
    def __iter__(self):
        return iter(self.vals)
    
    def __getitem__(self, index):
        return self.vals[index]
    
#     def __delitem__(self, index=None, arr=None):
#         if index:
#             arr = self[index]
        
#         if arr not in self:
#             print(f'{arr} not here')
#             return
        
#         arr = tuple(arr)
        
#         del 
        
#         s = self.tree
#         for a in arr[:-1]:
#             s = s[a]
#         del s[arr[-1]]
        
        
    def __contains__(self, arr):
        s = self.tree
        for a in arr:
            if not s or a not in s:
                return False
            s = s[a]
#         return 'END' in s
        return True
    
    def add(self, arr):
        if arr in self:
            return
        self.vals.append(tuple(arr))
        s = self.tree
        for a in arr:
            if a not in s:
                s[a] = {}
            s = s[a]
#         s['END'] = True
        
    def __str__(self):
        return '\n'.join(map(str,self.vals))

In [5]:
# this function takes in a, the sum of the values in a line, and b, the # of voltorbs in the line
#     and returns a Group of all of the arrangements that satisfy those conditions
# dont remember why the cache has a maxsize
@lru_cache(maxsize=36)
def getPossLines(a,b):
    g = Group()
    
    # special case for when theres only voltorbs
    if a == 0:
        g.add([0]*5)
#     elif a > 2*(5-b):
#         h = getPossLines(4*(5-b) - a, b)
#         for line in h:
#             g.add([(4-v)%4 for v in line])
    else:
        # this should be based on stars and bars
        # i believe it first decides where the voltorbs go
        #     and then fills in the rest of the values in between them
        zero_pos = [*cwr(range(5-b+1), b)]
        for i in cwr(range(a+b-5+1), 5-b-1):
            dd = np.diff([0, *i, a+b-5])
            if (dd > 2).any():
                continue
            dd += 1
            for j in zero_pos:
                g.add(np.insert(dd, j, 0))
    
    return g

In [9]:
# recursive function to find all possible solutions from the possible row and col options
def rec(i=0):
    if i == 4: #once we get to the last row
        # the last row is uniquely determined by the previous 4 since the sums need to match the total
        # this means we can calculate it directly
        brd[i] = cols[:,0] - brd[:i,:].sum(axis=0)
        
        # check if the calculated last row is possible and if each column has the right number of voltorbs
        if brd[i] in possRows[i] and ((brd == 0).sum(axis=0) == cols[:,1]).all():
            sols.append(np.copy(brd))
            print(len(sols), end='\r')
    else:
        if i:
            # only need to do this check when i > 0 since a blank board will always pass
            # for each column, check if the current prefix determined by the selected rows is valid
            #     if its not valid, then return and backtrack
            for j in range(5):
                if brd[:i,j] not in possCols[j]:
                    return

        # try each option for the current row
        for row in possRows[i]:
            brd[i] = row
            rec(i+1)

# with open("vflip.in") as f:
#     R = C = 5

#     rows = np.array([[*map(int, f.readline().split())] for _ in range(R)])
#     cols = np.array([[*map(int, f.readline().split())] for _ in range(C)])

path = '/Users/ssbhatt/Downloads/chromedriver'
# driver = webdriver.Chrome(executable_path=path)
driver = webdriver.Chrome()
driver.get('https://voltorbflip.brandon-stein.com/')

In [16]:
points = np.array([elem.text for elem in driver.find_element(by=By.CSS_SELECTOR,value="div[class*='points']")], dtype=int)
bombs = np.array([elem.text for elem in driver.find_element(by=By.CSS_SELECTOR,value="div[class*='bombs']")], dtype=int)
rows, cols = np.vsplit(np.column_stack((points, bombs)), 2)

print(rows)
print(cols)

brd = np.zeros((5,5), dtype=int)
possRows = [getPossLines(*row) for row in rows]
possCols = [getPossLines(*col) for col in cols]
# for pr in possRows:
#     print(pr)
#     print()

# find all the solutions
sols = []
%time rec()
# print(sols)
sols = np.stack(sols)
print(sols.shape)

# this is the actual answer scraped from the website for comparison purposes
np.array(['0'+elem.text for elem in driver.find_elements_by_css_selector("div[class*='tile-module--front']")],dtype=int).reshape((5,5))

TypeError: 'WebElement' object is not iterable

In [6]:
driver.close()

In [79]:
# this string is used with the add function above
# the setup is very jank but it works

# the characters are divided into group of 3 using python magic in gcond
conds = '122423'
gcond = map(add,zip(*[iter(conds)]*3))
# each of the vectors returned by add for each of the conditions are ANDed together 
log_and = reduce(lambda a,b: a & b, gcond, np.ones((sols.shape[0]),dtype=bool))
# the places in log_and that are True correspond to boards that satisfy all of the conditions in conds
# filter the rest of the boards out
sols2 = sols[log_and]
print(sols2.shape)

# the way you, as the user, use this is:
#     each time you click a square at position (i,j) and see value k
#     add 'ijk' to the end of conds and rerun the cell
# you prob only need to add conditions where there was a < 1 prob of the square being that value
#     since it is then able to eliminate some of the boards in consideration
  
# for each position in the board
for i in range(5):
    for j in range(5):
        # these track the value that has the highest probability so far and what that probability is
        maxp = 0
        maxv = 0

        # get all of the values that can be in the current position
        stk = sols2[:,i,j]
        
        # for each unique value
        for v in set(stk):
            # find the probability that a random valid board has that value in the current position
            p = (stk == v).sum() / stk.shape[0]
            
            # update the max variables
            if p >= maxp:
                maxp = p
                maxv = v
                
        # since we only care about finding 2s and 3s, we display only positions where either
        #     1. either 2 or 3 has the highest probability
        #     2. a voltorb cannot be in that position so it's safe to click
        if maxv > 1 or 0 not in stk:
            # the printed values are:
            # position, possible values, value with highest prob, the prob of that value if its < 1, the prob of 0
            # i print the last value to break ties where 2 positions have the same probability of being 2 or 3,
            #     but one has a lower prob of 0, making it safer to click
            print((i,j), set(stk), maxv, str(maxp)*(maxp<1), (stk==0).sum()/stk.shape[0] if 0 in stk else '')

    print()

(15, 5, 5)
(0, 1) {0, 1, 3} 3 0.4666666666666667 0.4
(0, 2) {0, 3} 3 0.5333333333333333 0.4666666666666667
(0, 3) {1} 1  

(1, 2) {2} 2  
(1, 3) {1} 1  

(2, 3) {1} 1  

(3, 1) {0, 1, 3} 3 0.5333333333333333 0.4
(3, 3) {1} 1  

(4, 2) {3} 3  
(4, 3) {1} 1  



<ipython-input-79-9e9394684f28>:48: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  print((i,j), set(stk), maxv, str(maxp)*(maxp<1), (stk==0).sum()/stk.shape[0] if 0 in stk else '')


In [46]:
# idk what this is 
%%prun
gpl = getPossLines   
for z in range(6):
    t = 4*(5-z)
    for i in range(5-z, 3*(5-z)+1):
        for a in gpl(i,z):
            assert ((4-b)%4 for b in a) in gpl(t-i, z)

In [48]:
# this is for testing
conds = '002010'
print(*zip(*[iter(conds)]*3))
gcond = [*map(add,zip(*[iter(conds)]*3))]
# print(gcond)
log_and = reduce(lambda a,b: a & b, gcond, np.ones((sols.shape[0]), dtype=bool))
# print(log_and)
sols2 = sols[log_and]
print(sols2.shape)

('0', '0', '2') ('0', '1', '0')
(0, 5, 5)
